# Here, we Extract, Transform, and Load the Data before going into Feature Engineering/Selection

As we found during the data exploration phase, our data is in good shape already, so most of the ETL pipeline is already complete. 

In this section, we will focus on feature selection, since that will be a primary challenge in creating a useful model.

We will combine the ***Receiver Operating Characteristic (ROC) curve, the Area Under Curve (AUC) and Mutual Information (MI) algorithm*** in order to find the top N features by importance. We will then optimize for N when building our Model in order to find the most accurate model using the least amount of features possible. 

Credit to https://machinelearningmastery.com/feature-selection-with-categorical-data/ from which the MI code was modified. 

## Begin by importing the bankruptcy data from Cloud Object Storage

In [3]:
# Import bankruptcy data from COS
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_1c720e1b65aa412b89762bf230a6b5f6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='7GN1Hk1VrQ0_RGdupGi9ZaphffKXQb6iMJPgYn1DNqgh',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_1c720e1b65aa412b89762bf230a6b5f6.get_object(Bucket='ibmcapstoneproject-donotdelete-pr-iiaol8yd9vtgm6',Key='data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,operating gross margin,realized sales gross margin,operating profit rate,tax Pre-net interest rate,after-tax net interest rate,non-industry income and expenditure/revenue,...,net income to total assets,total assets to GNP price,No-credit interval,Gross profit to Sales,Net income to stockholder's Equity,liability to equity,Degree of financial leverage (DFL),Interest coverage ratio( Interest expense to EBIT ),one if net income was negative for the last two year zero otherwise,equity to liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [4]:
# IMPORTANT: sklearn.__version__ must be >= 0.24.1 for calculations later down the line
# If your version does not meet this requirement, you must execute the following line and then RESTART KERNEL
#! pip install -U scikit-learn==0.24.1

## Next, plot the ROC curve

In [5]:
# Import necessary modules
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [6]:
# Some wrangling to prepare the data for analysis

# Import the dataset as a numpy array
data = df.to_numpy()

# Get feature names using Pandas DataFrame ... we'll need these later
feature_names = list(df.columns[1:])

# Break data into Features and label
X, y = data[:, 1:], data[:, 0]

# Check data shape to make sure everything lines up.
# We expect to see 6819 rows for both X and y
# We expect to see X having 95 dimensions (96 total dimensions, minus 1 for the Label column)
# We expect to see y having 1 dimension, as it is the Label column (i.e. 'Bankruptcy?')
print(feature_names)
print(X.shape, y.shape)

[' ROA(C) before interest and depreciation before interest', ' ROA(A) before interest and % after tax', ' ROA(B) before interest and depreciation after tax', ' operating gross margin', ' realized sales gross margin', ' operating profit rate', ' tax Pre-net interest rate', ' after-tax net interest rate', ' non-industry income and expenditure/revenue', ' continuous interest rate (after tax)', ' operating expense rate', ' research and development expense rate', ' cash flow rate', ' interest-bearing debt interest rate', ' tax rate (A)', ' per Net Share Value (B)', ' Net Value Per Share (A)', ' Net Value Per Share (C)', ' Persistent EPS in the Last Four Seasons', ' Cash Flow Per Share', ' Revenue Per Share (Yuan)', ' Operating Profit Per Share (Yuan)', ' Per Share Net profit before tax (yuan)', ' realized sales gross profit growth rate', ' operating profit growth rate', ' after-tax net profit growth rate', ' regular net profit growth rate', ' continuous net profit growth rate', ' total asse

In [7]:
# Now, time to plot the ROC curve

# This code takes awhile to run (10-15 mins with 2 CPU and 8GB of RAM), as there are a lot of features to analyze.
# As we will see, there is an easier, faster method. But this was still my first step, so I include it here. 

# IF YOU ARE RUNNING THIS NOTEBOOK FROM SCRATCH, I RECOMMEND SKIPPING THIS CELL ENTIRELY
    # there is a better approach in the cells below

"""

# Set dummy variable to determine which class to plot the ROC curve over
# It turns out that this doesn't matter, i.e. y=0 and y=1 have the same result, but we include it anyway so the code works

y_ = y == 0

plt.figure(figsize=(13,7))
for col in range(X.shape[1]):
    tpr,fpr = [],[]
    for threshold in np.linspace(min(X[:,col]), max(X[:,col]), 100):
        detP = X[:,col] < threshold
        tpr.append(sum(detP & y_) / sum(y_)) 
        fpr.append(sum(detP & (~y_)) / sum((~y_)))
        
    if auc(fpr,tpr) < 0.5:
        aux = tpr
        tpr = fpr
        fpr = aux
    
    
    plt.plot(fpr, tpr, label=feature_names[col] + ', auc = '\
                        + str(np.round(auc(fpr,tpr),decimals=3)))
    
plt.title('ROC curve - Bankruptcy features')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

"""

"\n\n# Set dummy variable to determine which class to plot the ROC curve over\n# It turns out that this doesn't matter, i.e. y=0 and y=1 have the same result, but we include it anyway so the code works\n\ny_ = y == 0\n\nplt.figure(figsize=(13,7))\nfor col in range(X.shape[1]):\n    tpr,fpr = [],[]\n    for threshold in np.linspace(min(X[:,col]), max(X[:,col]), 100):\n        detP = X[:,col] < threshold\n        tpr.append(sum(detP & y_) / sum(y_)) \n        fpr.append(sum(detP & (~y_)) / sum((~y_)))\n        \n    if auc(fpr,tpr) < 0.5:\n        aux = tpr\n        tpr = fpr\n        fpr = aux\n    \n    \n    plt.plot(fpr, tpr, label=feature_names[col] + ', auc = '                        + str(np.round(auc(fpr,tpr),decimals=3)))\n    \nplt.title('ROC curve - Bankruptcy features')\nplt.xlabel('False Positive Rate')\nplt.ylabel('True Positive Rate')\nplt.legend()\nplt.show()\n\n"

### If you run the code above, you'll see that there are far too many curves to pick out the most promising features.

So, we will take a different approach by choosing the top N features ranked by AUC. AUC is the integral of the ROC curve, so a higher score means more True Positives in the data. 

In [8]:
# This code is similar to the code for plotting, but here we simply record the AUC values
# This code takes approximately 7 mins to run using 2 CPU and 8GB RAM

y_ = y == 0 

auc_scores = []

for col in range(X.shape[1]):
    tpr, fpr = [], []
    for threshold in np.linspace(min(X[:,col]), max(X[:,col]), 100):
        detP = X[:,col] < threshold
        tpr.append(sum(detP & y_) / sum(y_)) 
        fpr.append(sum(detP & (~y_)) / sum((~y_)))
        
    if auc(fpr,tpr) < 0.5:
        aux = tpr
        tpr = fpr
        fpr = aux
        
    auc_value = str(np.round(auc(fpr, tpr), decimals=3))
    auc_scores.append([feature_names[col], auc_value])
    
    print(feature_names[col] + ', auc = ', auc_value)

 ROA(C) before interest and depreciation before interest, auc =  0.863
 ROA(A) before interest and % after tax, auc =  0.861
 ROA(B) before interest and depreciation after tax, auc =  0.865
 operating gross margin, auc =  0.717
 realized sales gross margin, auc =  0.716
 operating profit rate, auc =  0.501
 tax Pre-net interest rate, auc =  0.513
 after-tax net interest rate, auc =  0.538
 non-industry income and expenditure/revenue, auc =  0.614
 continuous interest rate (after tax), auc =  0.512
 operating expense rate, auc =  0.511
 research and development expense rate, auc =  0.538
 cash flow rate, auc =  0.705
 interest-bearing debt interest rate, auc =  0.509
 tax rate (A), auc =  0.741
 per Net Share Value (B), auc =  0.827
 Net Value Per Share (A), auc =  0.826
 Net Value Per Share (C), auc =  0.824
 Persistent EPS in the Last Four Seasons, auc =  0.882
 Cash Flow Per Share, auc =  0.684
 Revenue Per Share (Yuan), auc =  0.5
 Operating Profit Per Share (Yuan), auc =  0.798
 Pe

In [9]:
# Modify our AUC data to prepare it for further processing down the line
auc_scores = np.array(auc_scores)

auc = []
for x in range(auc_scores.shape[0]):
    auc.append([auc_scores[x, 0], np.float(auc_scores[x, 1])])

In [10]:
# Import auc into Pandas DataFrame for ranking
df_auc = pd.DataFrame(auc)

# Rename column headers
mapper = { 
    0 : 'Feature Name',
    1 : 'AUC Score'
}
df_auc = df_auc.rename(columns=mapper)

df_auc.head()

,Feature Name,AUC Score
0,ROA(C) before interest and depreciation befor...,0.863
1,ROA(A) before interest and % after tax,0.861
2,ROA(B) before interest and depreciation after...,0.865
3,operating gross margin,0.717
4,realized sales gross margin,0.716


In [11]:
# At this point we have what we need to move forward. But if we wanted to stop here and just rank by AUC, here's what we would have

#Select top N features by AUC and export them to a list named 'features_auc'

N = 10

df_auc.nlargest(N, 'AUC Score', keep='first')

# Finally, here is the list of most important features determining bankrupty, ranked by AUC

#print("The most important " + str(N) + " features, ranked by AUC:")
#for m in range(len(features_auc)):
#    print("No. " + str(m+1) + ":" + features_auc[m, 0] + ', AUC = ' + str(features_auc[m, 1]))

,Feature Name,AUC Score
18,Persistent EPS in the Last Four Seasons,0.882
85,net income to total assets,0.877
42,net profit before tax/paid-in capital,0.876
22,Per Share Net profit before tax (yuan),0.873
2,ROA(B) before interest and depreciation after...,0.865
0,ROA(C) before interest and depreciation befor...,0.863
1,ROA(A) before interest and % after tax,0.861
67,Retained Earnings/Total assets,0.858
36,debt ratio %,0.854
37,net worth/assets,0.854


With N=10, each of the top 10 features ranked by AUC is data we could likely find on any public company. So we are looking good so far.

## Next, we move onto the Mutual Information algorithm

The theory behind MI is to compare each feature to determine whether or not certain features contribute the same amount of information to the overall model. The idea is that if, for example, 10 features out of 20 total features contribute similar amounts of information, we should simplify the model by reducing the 10 similar features to, say, 1 feature, thereby reducing our total feature count to 11. 

You can read more here: https://machinelearningmastery.com/information-gain-and-mutual-information/

In [12]:
# Import necessary modules

from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from matplotlib import pyplot
import numpy as np
import sklearn

In [13]:
# Define functions to ease extraction and preparation of data for feature engineering

#body = filename

def load_dataset(filename):
    data = pd.read_csv(filename)
    dataset = data.values
    X = dataset[:, 1:]
    y = dataset[:,0]
    
    X = X.astype(str)
    return X, y 

def prepare_inputs(X_train, X_test):
    oe = OrdinalEncoder()
    
    oe.fit(X_train)
    X_train_enc = oe.transform(X_train)
    
    oe.fit(X_test) ## IMPORTANT, if you don't use oe.fit on both X_train and X_test the function breaks
    X_test_enc = oe.transform(X_test)
    return X_train_enc, X_test_enc

def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc

def select_features(X_train, y_train, X_test):
    fs = SelectKBest(score_func = mutual_info_classif, k='all')
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

In [14]:
# Perform calculations to get scores

X, y = load_dataset(client_1c720e1b65aa412b89762bf230a6b5f6.get_object(Bucket='ibmcapstoneproject-donotdelete-pr-iiaol8yd9vtgm6',Key='data.csv')['Body'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
X_train_fs, X_test_fs, fs = select_features(X_train_enc, y_train_enc, X_test_enc)

In [15]:
# Print scores for the features. Higher is better

for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))

Feature 0: 0.032125
Feature 1: 0.039238
Feature 2: 0.035258
Feature 3: 0.016897
Feature 4: 0.015375
Feature 5: 0.019606
Feature 6: 0.031128
Feature 7: 0.031471
Feature 8: 0.025675
Feature 9: 0.036402
Feature 10: 0.003413
Feature 11: 0.001147
Feature 12: 0.012948
Feature 13: 0.012377
Feature 14: 0.017541
Feature 15: 0.038551
Feature 16: 0.037118
Feature 17: 0.035629
Feature 18: 0.041010
Feature 19: 0.007480
Feature 20: 0.001656
Feature 21: 0.023957
Feature 22: 0.041434
Feature 23: 0.009369
Feature 24: 0.004865
Feature 25: 0.015141
Feature 26: 0.012337
Feature 27: 0.011277
Feature 28: 0.019068
Feature 29: 0.037106
Feature 30: 0.013398
Feature 31: 0.002253
Feature 32: 0.023513
Feature 33: 0.024122
Feature 34: 0.034903
Feature 35: 0.033463
Feature 36: 0.032669
Feature 37: 0.033885
Feature 38: 0.011111
Feature 39: 0.037038
Feature 40: 0.000000
Feature 41: 0.022015
Feature 42: 0.040808
Feature 43: 0.007901
Feature 44: 0.005966
Feature 45: 0.001414
Feature 46: 0.001064
Feature 47: 0.001109
Fe

In [16]:
# Put final results into new Pandas DataFrame and rename column headers

#df = pd.read_csv(filename)

df_mi = []

for col in range(1, len(feature_names)):
    df_mi.append([df.columns[col], fs.scores_[col-1]])
    
df_mi = pd.DataFrame(df_mi)

mapping = {
    0 : 'Feature Name',
    1 : 'MI Score'
}
df_mi = df_mi.rename(columns = mapping)
df_mi.head()

,Feature Name,MI Score
0,ROA(C) before interest and depreciation befor...,0.032125
1,ROA(A) before interest and % after tax,0.039238
2,ROA(B) before interest and depreciation after...,0.035258
3,operating gross margin,0.016897
4,realized sales gross margin,0.015375


In [17]:
# Show top 10 results by Score
N = 10
df_mi.nlargest(N, 'MI Score', keep='first').head()

,Feature Name,MI Score
89,Net income to stockholder's Equity,0.044734
22,Per Share Net profit before tax (yuan),0.041434
18,Persistent EPS in the Last Four Seasons,0.041010
42,net profit before tax/paid-in capital,0.040808
1,ROA(A) before interest and % after tax,0.039238


## Finally, we put it all together, combining AUC and MI to find the best N features

In [18]:
df_auc

,Feature Name,AUC Score
0,ROA(C) before interest and depreciation befor...,0.863
1,ROA(A) before interest and % after tax,0.861
2,ROA(B) before interest and depreciation after...,0.865
3,operating gross margin,0.717
4,realized sales gross margin,0.716
...,...,...
90,liability to equity,0.764
91,Degree of financial leverage (DFL),0.525
92,Interest coverage ratio( Interest expense to E...,0.525
93,one if net income was negative for the last tw...,0.000


In [19]:
# Merge our datasets
features_auc_mi = pd.merge(df_auc, df_mi, on='Feature Name')
features_auc_mi.head()

# We give more weight to the MI Score to even it out with AUC Score
# Basically we want both scores to fall within the same order of magnitude (10^(-1))
features_auc_mi['mi_score_mod'] = features_auc_mi['MI Score']*10

# Calculate the average score of AUC and modified MI score
features_auc_mi['avg_score'] = (features_auc_mi['AUC Score'] + features_auc_mi['mi_score_mod']) / 2

# Create a copy of features_auc_mi for naming conventions
features_final = features_auc_mi
"""
This was a relic from a past workflow. In reality, it makes the most sense to choose N while training models, not during feature selection.

# Create the final DataFrame containing the top N features
# features_final = features_auc_mi.nlargest(N, 'avg_score', keep='first')
"""
# View the features dataframe to see most important features in our dataset
features_final.head()

,Feature Name,AUC Score,MI Score,mi_score_mod,avg_score
0,ROA(C) before interest and depreciation befor...,0.863,0.032125,0.321250,0.592125
1,ROA(A) before interest and % after tax,0.861,0.039238,0.392376,0.626688
2,ROA(B) before interest and depreciation after...,0.865,0.035258,0.352580,0.608790
3,operating gross margin,0.717,0.016897,0.168974,0.442987
4,realized sales gross margin,0.716,0.015375,0.153747,0.434873


# Save selected features DataFrame to Cloud Object Storage

In [ ]:
# Install pyspark and ibmos2spark

! pip install pyspark
! pip install ibmos2spark

# Define SparkContext and SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [21]:
# Save data to project folder

"""

NOTE:
================================================
 
For the line: project = Project(sc, "939d834e-ff85-4a5d-9952-9bf94243d5ae", "p-dd6b8c4af5c50be232dc2d685e8091eb69302d20")

the Project() function is in the form Project(sc, <my_project_id>, <my_token_id>)

See: https://www.ibm.com/support/pages/how-do-you-save-dataframe-watson-studio-project-notebook-asset-cloud-object-storage 
to learn how to find values for <my_project_id> and <my_project_token>


Final output of this cell should be something like:

{'file_name': 'data.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmcapstoneproject-donotdelete-pr-iiaol8yd9vtgm6',
 'asset_id': '1238f15e-8bb8-4a76-8cac-7beaf0917b86'}
 
 """

from project_lib import Project

project = Project(sc, "939d834e-ff85-4a5d-9952-9bf94243d5ae", "p-dd6b8c4af5c50be232dc2d685e8091eb69302d20")
project.save_data(file_name = "features_final.csv", data = features_final.to_csv(index=False), overwrite=True)

{'file_name': 'features_final.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmcapstoneproject-donotdelete-pr-iiaol8yd9vtgm6',
 'asset_id': 'a26a8a54-8736-4409-8cf5-8cdf04af8231'}